# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [2]:
from pyspark.sql import SparkSession



In [3]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
spark = SparkSession.builder.appName('ship').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/17 22:48:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
df = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [5]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



23/04/17 17:25:11 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
df1 = pd.read_csv('cruise_ship_info.csv')

In [ ]:
df1.isnull().mean()

In [ ]:
df1

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1.describe()

In [ ]:
num_values = [col for col in df1.columns if df1[col].dtype != 'O' and col != 'crew']
cat_var = [col for col in df1.columns if df1[col].dtype == "O"]
target = 'crew'

In [ ]:
num_values

In [ ]:
df1[num_values].hist()

In [ ]:
import seaborn as sns

corr = df1.corr()
sns.heatmap(corr)

In [ ]:
def age_group(x):
    
    if x <=10:
        return "child"
    elif x <=20:
        return 'young adult'
    else:
        return 'adult'

In [ ]:
df1['age_group'] = df1['Age'].apply(lambda x: age_group(x))

In [ ]:
df1.head()

In [ ]:
plt.scatter(df1['Age'],df1['crew'], marker = '.')
plt.xlabel('Age')
plt.ylabel('crew')

In [ ]:
df1.groupby('age_group').agg({"crew":'mean'})

In [ ]:
df1.groupby('age_group').apply(lambda x:x[num_values].mean())

In [ ]:
def function_cols(col1,col2):
    
    if col1 < col2:
        return "{} is greater than {}".format(col1,col2)
    else:
        return "{} is greater than {}".format(col2,col1)

In [ ]:
df1.head()

In [ ]:
df1['another_col'] = df1.apply(lambda x : function_cols(x['length'],x['cabins']), axis=1)

In [ ]:
df1.head()

In [ ]:
df.printSchema()

In [6]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(df).transform(df)


In [7]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

assembler = VectorAssembler(
  inputCols=['Age',
             'Tonnage',
             'passengers',
             'length',
             'cabins',
             'passenger_density',
             'cruise_cat'],
    outputCol="features")

In [8]:
output = assembler.transform(indexed)

In [9]:
output.select("features", "crew").show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [10]:
final_data = output.select("features", "crew")

In [11]:
train_data, test_data = final_data.randomSplit([.7,.3])

In [13]:
lr = LinearRegression(labelCol='crew')

In [14]:
lr_model = lr.fit(train_data)

23/04/17 17:25:45 WARN Instrumentation: [a83724ac] regParam is zero, which might cause numerical instability and overfitting.
23/04/17 17:25:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/04/17 17:25:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/04/17 17:25:46 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [15]:
summary = lr_model.summary

In [16]:
summary.r2

0.964491072994055

In [17]:
summary.rootMeanSquaredError

0.6523786858991226

In [19]:
lr_model.coefficients

DenseVector([-0.0089, 0.0243, -0.1678, 0.4187, 0.773, -0.0066, 0.0387])

In [20]:
lr_model.intercept

-0.8498143926067289

In [21]:
summary.meanSquaredError

0.4255979498154661

In [25]:
df.select("crew").describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [12]:
df.filter(df['Age'] <10).agg({'crew':'mean'}).show()

+------------------+
|         avg(crew)|
+------------------+
|10.667000000000002|
+------------------+



In [14]:
df.filter(df['Age'] <10).groupBy('Age').agg({"crew":'mean'}).show()

+---+------------------+
|Age|         avg(crew)|
+---+------------------+
|  6|           9.39875|
|  5|            10.726|
|  9|10.311999999999998|
|  4|              21.0|
|  8|10.533333333333333|
|  7|11.823333333333332|
+---+------------------+



In [15]:
temp = df.createOrReplaceTempView("Person")

In [17]:
spark.sql("Select * from Person").show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 